# OpenEO

For this tutorial we will use the openEO instance of Copernicus Data Space Ecosystem. You will need a CDSE account to use this tutorial. Account can be created at [CDSE](https://dataspace.copernicus.eu/) free of charge. 
Note that the code snippets in this guide work the same way for the other back-ends listed in the [openEO Hub](https://hub.openeo.org/). 
Just the collection identifier and band names might differ.

Establish a connection to the back-end. 

In [ ]:
import openeo

connection = openeo.connect("https://openeo.dataspace.copernicus.eu/openeo/1.2").authenticate_oidc()

## Creating a datacube

First we need to define a slice of a data collection we want to use. Then we can manipulate the datacube by applying openEO processes and save the result at the end. By applying an openEO process on a datacube, we create a new datacube object that represents the manipulated data. 

In [ ]:
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 10.05, "south": 55.21, "east": 10.1, "north": 55.23},
    temporal_extent=["2024-05-01", "2024-05-30"],
    bands=['B02', 'B03', 'B04', 'B08'],
)

red = datacube.band('B04')
nir = datacube.band("B08")
# perform operation using feature variables (e.g. calculation of NDVI (normalized difference vegetation index))
datacube = (nir - red) / (nir + red)
# reduce on temporal dimension with max operator
datacube = datacube.max_time()
# provide result as geotiff image
datacube = datacube.save_result(format="gtiff")
print(datacube.to_json())

It's important to note that all the datacube processes we applied up to this point are not actually executed yet, neither locally nor remotely on the back-end. We just built an abstract representation of the algorithm (input data and processing chain), encapsulated in a local Datacube object. To trigger an actual execution (on the back-end) we have to explicitly send this representation to the back-end.

In [ ]:
job = datacube.create_job()

Launch processing of submitted batch job

In [ ]:

if job.job_id:
    print(job.job_id)
    print(job.start_and_wait())
    print (job.describe_job())
else:
    print("Job ID is None")

To obtain results and download files

In [ ]:
if job.job_id:
    job.get_results().download_files("output")

Visualize your result

In [ ]:
import openeo
import rasterio
import matplotlib.pyplot as plt

%matplotlib inline


with rasterio.open("output/openEO.tif") as ds:
    data = ds.read(1)

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(14, 6))
im = ax1.imshow(data, vmin=0, vmax=1)
plt.colorbar(im, ax=ax1, orientation="horizontal")
ax1.set_title("NDVI Map")
ax2.hist(data.flat, bins=32)
ax2.set_title("NDVI Histogram")
ax2.set_xlabel("NDVI value")
ax2.set_ylabel("frequency");